### Python Extract Transform Load Example

In [14]:
# Imports

import requests
import pandas as pd
from sqlalchemy import create_engine

In [15]:
# Extraction

def extract() -> dict:
    """
        This API estracts data from 
        http://universities.hipolabs.com
    """
    
    API_URL = "http://universities.hipolabs.com/search?country=United+States"
    data = requests.get(API_URL).json()
    return data


In [16]:
data = extract()
print(data)

[{'state-province': None, 'domains': ['marywood.edu'], 'country': 'United States', 'web_pages': ['http://www.marywood.edu'], 'name': 'Marywood University', 'alpha_two_code': 'US'}, {'state-province': None, 'domains': ['lindenwood.edu'], 'country': 'United States', 'web_pages': ['http://www.lindenwood.edu/'], 'name': 'Lindenwood University', 'alpha_two_code': 'US'}, {'state-province': None, 'domains': ['sullivan.edu'], 'country': 'United States', 'web_pages': ['https://sullivan.edu/'], 'name': 'Sullivan University', 'alpha_two_code': 'US'}, {'state-province': None, 'domains': ['fscj.edu'], 'country': 'United States', 'web_pages': ['https://www.fscj.edu/'], 'name': 'Florida State College at Jacksonville', 'alpha_two_code': 'US'}, {'state-province': None, 'domains': ['xavier.edu'], 'country': 'United States', 'web_pages': ['https://www.xavier.edu/'], 'name': 'Xavier University', 'alpha_two_code': 'US'}, {'state-province': None, 'domains': ['tusculum.edu'], 'country': 'United States', 'web

In [17]:
# Transformation

def transform(data: dict) -> pd.DataFrame:
    """
        Transform the dataset into desired structure and filters
    """
    
    # Put the data into a pandas dataframe
    df = pd.DataFrame(data)
    print(f"Total number of universities from API {len(data)}")
    
    # Filter just for California universities
    df = df[df['name'].str.contains('California')]
    print(f"Number of universities in California {len(df)}")
    
    # formating all domains and web_pages separeted by coma
    df['domains']  = [','.join(map(str, l)) for l in df['domains']]
    df['web_pages']  = [','.join(map(str, l)) for l in df['web_pages']]
    
    # Reseting the index because of the filtering
    df = df.reset_index(drop= True)
    
    return df[['domains', 'country', 'web_pages', 'name']]

In [18]:
df = transform(data)

Total number of universities from API 4562
Number of universities in California 84


In [21]:
# Load

def load(df: pd.DataFrame) -> None:
    """
        Loads data into sqllite database
    """
    
    # Create an sqlite engine
    disk_engine = create_engine('sqlite:///data/my_lite_storage.db')
    
    # Write down df's data to a sqlite file
    df.to_sql('cal_uni', disk_engine, if_exists='replace')

In [22]:
load(df)